In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd
import pyarrow.parquet as pq
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score, classification_report
import bisect
from sklearn.metrics import f1_score
from IPython.display import clear_output
import sklearn.metrics as m

In [2]:
s = '12345'
print(s[1:-2])

23


In [3]:
data = dd.read_parquet('Dataset\\figma_plan\\full_gender_data_txt.parquet').compute()

data = data.loc[:,data.columns[:-3].to_list()]
# добавляем target, удаляем nan
data = data.merge(pd.read_parquet('public_train.pqt',columns=['user_id','age'])[['age','user_id']], on = 'user_id', how = 'inner')

data = data.loc[~(data['age'].isna()) & (data['age'] != 'NA')]
data['age'] = data['age'].astype('int8')

display(data.head())
print(data.info(verbose=True, show_counts=True))

,user_id,region_name,region_count,city_name,city_count,cpe_manufacturer_name,cpe_type_cd,price,holyday_fraction,morning_fraction,...,megapteka-ru.turbopages.org,127.0.0.1,bukvaprava.ru,belnovosti-by.turbopages.org,ngs24-ru.turbopages.org,forum.mfd.ru,yomed.ru,7info.ru,other_url,age
0,0,Москва,1,Москва,1,Samsung,smartphone,2990.0,0.243523,0.119,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031088,35
1,1,Москва,3,Москва,6,Xiaomi,smartphone,NaN,0.200573,0.323,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013372,41
2,2,Республика Коми,1,Печора,1,Huawei,smartphone,5915.0,0.299270,0.187,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002433,25
3,3,Воронежская область,1,Воронеж,1,Huawei Device Company Limited,smartphone,13990.0,0.181818,0.178,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010909,50
4,4,Краснодарский край,5,Анапа,9,Huawei,smartphone,12990.0,0.326898,0.372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011583,28


<class 'pandas.core.frame.DataFrame'>
Int64Index: 269999 entries, 0 to 269999
Data columns (total 3014 columns):
 #     Column                                            Non-Null Count   Dtype  
---    ------                                            --------------   -----  
 0     user_id                                           269999 non-null  int32  
 1     region_name                                       269999 non-null  object 
 2     region_count                                      269999 non-null  int8   
 3     city_name                                         269999 non-null  object 
 4     city_count                                        269999 non-null  int8   
 5     cpe_manufacturer_name                             269999 non-null  object 
 6     cpe_type_cd                                       269999 non-null  object 
 7     price                                             263050 non-null  float32
 8     holyday_fraction                                  269999 non

In [4]:
type_file = 'train' #train
file_name = ['cpe_type_cd', 'url_host', 'part_of_day']
path_age_data = 'Dataset\\age_data\\'

for file in file_name:
    buf_data = dd.read_parquet(path_age_data + f'basket_prob_{file}_{type_file}.parquet').compute()
    print(buf_data.info())
    buf_data = buf_data.rename(columns={'bask_0': 'bask_0' + file, 'bask_1': 'bask_1' + file, 'bask_2': 'bask_2' + file, 'bask_3': 'bask_3' + file, 'bask_4': 'bask_4' + file, 'bask_5': 'bask_5' + file})

    if file =='url_host':
        data = data.merge(buf_data, how='left', on='user_id')
    else:
        data = data.merge(buf_data, how='inner', on='user_id')
    del buf_data


data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269999 entries, 0 to 269998
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   bask_0   269999 non-null  float64
 1   bask_1   269999 non-null  float64
 2   bask_2   269999 non-null  float64
 3   bask_3   269999 non-null  float64
 4   bask_4   269999 non-null  float64
 5   bask_5   269999 non-null  float64
 6   user_id  269999 non-null  int64  
dtypes: float64(6), int64(1)
memory usage: 14.4 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269999 entries, 0 to 269998
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   bask_0   269999 non-null  float64
 1   bask_1   269999 non-null  float64
 2   bask_2   269999 non-null  float64
 3   bask_3   269999 non-null  float64
 4   bask_4   269999 non-null  float64
 5   bask_5   269999 non-null  float64
 6   user_id  269999 non-null  int64  
dtypes: float64(6), in

In [5]:
data.to_parquet('Dataset\\age_data\\data_train.parquet', index=False)
data.info(verbose=True)
del data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269999 entries, 0 to 269998
Data columns (total 3032 columns):
 #     Column                                            Dtype  
---    ------                                            -----  
 0     user_id                                           int32  
 1     region_name                                       object 
 2     region_count                                      int8   
 3     city_name                                         object 
 4     city_count                                        int8   
 5     cpe_manufacturer_name                             object 
 6     cpe_type_cd                                       object 
 7     price                                             float32
 8     holyday_fraction                                  float32
 9     morning_fraction                                  float32
 10    day_fraction                                      float32
 11    evening_fraction                   

### Teст

In [6]:
data = dd.read_parquet('Dataset\\figma_plan\\full_gender_data_txt.parquet').compute()

data = data.loc[:,data.columns[:-3].to_list()]

display(data.head())
print(data.info(verbose=True, show_counts=True))

,user_id,region_name,region_count,city_name,city_count,cpe_manufacturer_name,cpe_type_cd,price,holyday_fraction,morning_fraction,...,psy-magic.org,megapteka-ru.turbopages.org,127.0.0.1,bukvaprava.ru,belnovosti-by.turbopages.org,ngs24-ru.turbopages.org,forum.mfd.ru,yomed.ru,7info.ru,other_url
0,0,Москва,1,Москва,1,Samsung,smartphone,2990.0,0.243523,0.119,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031088
1,1,Москва,3,Москва,6,Xiaomi,smartphone,NaN,0.200573,0.323,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013372
2,2,Республика Коми,1,Печора,1,Huawei,smartphone,5915.0,0.299270,0.187,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002433
3,3,Воронежская область,1,Воронеж,1,Huawei Device Company Limited,smartphone,13990.0,0.181818,0.178,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010909
4,4,Краснодарский край,5,Анапа,9,Huawei,smartphone,12990.0,0.326898,0.372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011583


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Data columns (total 3013 columns):
 #     Column                                            Non-Null Count   Dtype  
---    ------                                            --------------   -----  
 0     user_id                                           415317 non-null  int32  
 1     region_name                                       415317 non-null  object 
 2     region_count                                      415317 non-null  int8   
 3     city_name                                         415317 non-null  object 
 4     city_count                                        415317 non-null  int8   
 5     cpe_manufacturer_name                             415317 non-null  object 
 6     cpe_type_cd                                       415317 non-null  object 
 7     price                                             404563 non-null  float32
 8     holyday_fraction                                  415317 non

In [7]:
type_file = 'test' #train
file_name = ['cpe_type_cd', 'url_host', 'part_of_day']
path_age_data = 'Dataset\\age_data\\'

for file in file_name:
    buf_data = dd.read_parquet(path_age_data + f'basket_prob_{file}_{type_file}.parquet').compute()
    print(buf_data.info())
    buf_data = buf_data.rename(columns={'bask_0': 'bask_0' + file, 'bask_1': 'bask_1' + file, 'bask_2': 'bask_2' + file, 'bask_3': 'bask_3' + file, 'bask_4': 'bask_4' + file, 'bask_5': 'bask_5' + file})

    if file =='url_host':
        data = data.merge(buf_data, how='left', on='user_id')
    else:
        data = data.merge(buf_data, how='inner', on='user_id')
    del buf_data


data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144724 entries, 0 to 144723
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   bask_0   144724 non-null  float64
 1   bask_1   144724 non-null  float64
 2   bask_2   144724 non-null  float64
 3   bask_3   144724 non-null  float64
 4   bask_4   144724 non-null  float64
 5   bask_5   144724 non-null  float64
 6   user_id  144724 non-null  int64  
dtypes: float64(6), int64(1)
memory usage: 7.7 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144723 entries, 0 to 144722
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   bask_0   144723 non-null  float64
 1   bask_1   144723 non-null  float64
 2   bask_2   144723 non-null  float64
 3   bask_3   144723 non-null  float64
 4   bask_4   144723 non-null  float64
 5   bask_5   144723 non-null  float64
 6   user_id  144723 non-null  int64  
dtypes: float64(6), int

In [8]:

data.iloc[:, 3019:] = data.iloc[:, 3019:].fillna(value = 0.16)
#data = data.fillna(value = 0.16)
print(len(data['user_id'].unique()))
data.info(verbose=True, show_counts=True)

144724
<class 'pandas.core.frame.DataFrame'>
Int64Index: 144724 entries, 0 to 144723
Data columns (total 3031 columns):
 #     Column                                            Non-Null Count   Dtype  
---    ------                                            --------------   -----  
 0     user_id                                           144724 non-null  int32  
 1     region_name                                       144724 non-null  object 
 2     region_count                                      144724 non-null  int8   
 3     city_name                                         144724 non-null  object 
 4     city_count                                        144724 non-null  int8   
 5     cpe_manufacturer_name                             144724 non-null  object 
 6     cpe_type_cd                                       144724 non-null  object 
 7     price                                             140923 non-null  float32
 8     holyday_fraction                                  144

In [10]:
data.to_parquet('Dataset\\age_data\\data_test.parquet', index=False)
del data

#### Предсказание по возрасту

In [11]:
data = dd.read_parquet('Dataset\\age_data\\data_train.parquet', index=False).compute()

In [12]:
def age_bucket(x):
    return bisect.bisect_left([25,35,45,55,65], x)
print(age_bucket(67))

5


In [13]:
target = data['age'].map(age_bucket)
features = data.drop(columns=['age', 'user_id'])
del data

features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.33, random_state=12345)

In [14]:
list(features_train.dtypes[features_train.dtypes == 'category'].index)

[]

In [15]:
pool_train = Pool(features_train, target_train,
                  cat_features = list(features_train.select_dtypes(include=['object']).columns))

cat = CatBoostClassifier(task_type = 'GPU', random_seed = 12345)
cat.fit(pool_train, verbose=True)


Learning rate set to 0.187413
0:	learn: 1.6752772	total: 219ms	remaining: 3m 38s
1:	learn: 1.6000329	total: 385ms	remaining: 3m 12s
2:	learn: 1.5443257	total: 543ms	remaining: 3m
3:	learn: 1.5038684	total: 704ms	remaining: 2m 55s
4:	learn: 1.4738542	total: 847ms	remaining: 2m 48s
5:	learn: 1.4493130	total: 1.02s	remaining: 2m 48s
6:	learn: 1.4292387	total: 1.16s	remaining: 2m 44s
7:	learn: 1.4137608	total: 1.31s	remaining: 2m 42s
8:	learn: 1.4007885	total: 1.46s	remaining: 2m 40s
9:	learn: 1.3896160	total: 1.59s	remaining: 2m 37s
10:	learn: 1.3809363	total: 1.75s	remaining: 2m 37s
11:	learn: 1.3734850	total: 1.9s	remaining: 2m 36s
12:	learn: 1.3672035	total: 2.05s	remaining: 2m 35s
13:	learn: 1.3617423	total: 2.19s	remaining: 2m 34s
14:	learn: 1.3566245	total: 2.33s	remaining: 2m 33s
15:	learn: 1.3516864	total: 2.47s	remaining: 2m 31s
16:	learn: 1.3483445	total: 2.59s	remaining: 2m 29s
17:	learn: 1.3452568	total: 2.71s	remaining: 2m 27s
18:	learn: 1.3421805	total: 2.86s	remaining: 2m 2

In [16]:
#print(classification_report(target_train, cat.predict(features_train),target_names = ['<18', '18-25','25-34', '35-44', '45-54', '55-65', '65+']))
# 0.92 test
print(f"2 * weightF1 train = {2*f1_score(target_train, cat.predict(features_train), average='weighted')}")
print(f"2 * weightF1 test = {2*f1_score(target_valid, cat.predict(features_valid), average='weighted')}")

2 * weightF1 train = 1.0884796828552545
2 * weightF1 test = 0.9202719025388201


In [17]:
print(m.classification_report(target_valid, cat.predict(features_valid), target_names = ['19-25','26-34', '36-44', '46-54', '56-65', '66+']))

              precision    recall  f1-score   support

       19-25       0.58      0.40      0.48     11087
       26-34       0.52      0.64      0.57     28826
       36-44       0.43      0.53      0.48     25535
       46-54       0.40      0.26      0.32     14097
       56-65       0.42      0.25      0.32      7761
         66+       0.46      0.03      0.06      1794

    accuracy                           0.47     89100
   macro avg       0.47      0.35      0.37     89100
weighted avg       0.47      0.47      0.46     89100



In [10]:
cat_imp = pd.Series(cat.get_feature_importance(), index=features.columns).sort_values(ascending=False)
list(cat_imp.index)

['vk.com',
 'instagram.com',
 'online.sberbank.ru',
 'l.instagram.com',
 'bask_6url_host',
 'esia.gosuslugi.ru',
 'ad.adriver.ru',
 'sun9-29.userapi.com',
 'syndication.realsrv.com',
 'whatsapp.com',
 'i.ytimg.com',
 'hh.ru',
 'sun9-17.userapi.com',
 'sun9-5.userapi.com',
 'googleads.g.doubleclick.net',
 'id.tinkoff.ru',
 'ads.adfox.ru',
 'zen.yandex.ru',
 'bask_2url_host',
 'bask_0url_host',
 'm.zen.yandex.ru',
 'avatars.mds.yandex.net',
 'm.ok.ru',
 'ad.mail.ru',
 'm.avito.ru',
 't.mail.ru',
 'st.mycdn.me',
 's0.2mdn.net',
 'google.com',
 'mail.yandex.ru',
 'sun9-61.userapi.com',
 'away.vk.com',
 'sun9-27.userapi.com',
 'apple.com',
 'smi2.ru',
 'yandex.ru',
 'yastatic.net',
 's3.amazonaws.com',
 'other_url',
 'tpc.googlesyndication.com',
 'ru.m.wikipedia.org',
 'o2.mail.ru',
 'bask_4url_host',
 'm.vk.com',
 'play.google.com',
 'rb.infox.sg',
 'm.facebook.com',
 'frontend.vh.yandex.ru',
 'i.pinimg.com',
 'node3.online.sberbank.ru',
 'otvet.mail.ru',
 'online.vtb.ru',
 'baza.drom.ru',

In [14]:
pred = cat.predict(features_valid)
pred

array([[3],
       [2],
       [3],
       ...,
       [3],
       [4],
       [2]], dtype=int64)